## Data Transformation

In [3]:
import os
%pwd

'd:\\MLOPs\\End to end NLP Project with HuggingFace and Transformers\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\MLOPs\\End to end NLP Project with HuggingFace and Transformers'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [7]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [9]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

c:\Users\Admin\anaconda3\envs\ml_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Transformation Componenet

In [ ]:
# class DataTransformation:
#     def __init__(self, config):
#         """
#         Args:
#             config (DataTransformationConfig): Configuration object containing tokenizer info
#         """
#         self.config = config
#         self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        

#     def convert_examples_to_features(self, example_batch):
#         """
#         Convert a batch of examples (dialogue + summary) into tokenized input_ids, attention_mask, and labels.

#         Args:
#             example_batch (dict): Dictionary containing 'dialogue' and 'summary' lists

#         Returns:
#             dict: Dictionary with tokenized 'input_ids', 'attention_mask', and 'labels'
#         """
#         # Tokenize input dialogue
#         input_encodings = self.tokenizer(
#             example_batch['dialogue'],
#             max_length=self.config.max_input_length,
#             padding='max_length',
#             truncation=True
#         )

#         # Tokenize summary
#         target_encodings = self.tokenizer(
#             example_batch['summary'],
#             max_length=self.config.max_target_length,
#             padding='max_length',
#             truncation=True
#         )

#         return {
#             'input_ids': input_encodings['input_ids'],
#             'attention_mask': input_encodings['attention_mask'],
#             'labels': target_encodings['input_ids']
#         }
#     def convert(self):
#         dataset_samsum=load_from_disk(self.config.data_path)
#         dataset_samsum_pt=dataset_samsum.map(self.convert_examples_to_features,batched=True)
#         dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
        


In [16]:
import os
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer

# ------------------------------
# Configuration class
# ------------------------------
class DataTransformationConfig:
    def __init__(self):
        # Tokenizer and dataset paths
        self.tokenizer_name = "google/pegasus-cnn_dailymail"
        self.data_path = "artifacts/data_ingestion/samsum_dataset"
        self.root_dir = "artifacts/data_transformation"

        # Tokenization settings
        self.max_input_length = 512     # maximum length for dialogues
        self.max_target_length = 128    # maximum length for summaries

# ------------------------------
# Data Transformation class
# ------------------------------
class DataTransformation:
    def __init__(self, config):
        """
        Args:
            config (DataTransformationConfig): Configuration object containing tokenizer info
        """
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        """
        Convert a batch of examples (dialogue + summary) into tokenized input_ids, attention_mask, and labels.

        Args:
            example_batch (dict): Dictionary containing 'dialogue' and 'summary' lists

        Returns:
            dict: Dictionary with tokenized 'input_ids', 'attention_mask', and 'labels'
        """
        # Tokenize input dialogue
        input_encodings = self.tokenizer(
            example_batch['dialogue'],
            max_length=self.config.max_input_length,
            padding='max_length',
            truncation=True
        )

        # Tokenize summary
        target_encodings = self.tokenizer(
            example_batch['summary'],
            max_length=self.config.max_target_length,
            padding='max_length',
            truncation=True
        )

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        # If dataset directory does not exist, download SAMSum
        if not os.path.exists(self.config.data_path):
            print(f"Dataset not found at {self.config.data_path}. Downloading SAMSum dataset...")
            dataset_samsum = load_dataset("samsum")
            os.makedirs(self.config.data_path, exist_ok=True)
            dataset_samsum.save_to_disk(self.config.data_path)
            print("Dataset downloaded and saved locally.")

        # Load dataset from disk
        dataset_samsum = load_from_disk(self.config.data_path)

        # Convert examples to tokenized features
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)

        # Save processed dataset
        save_path = os.path.join(self.config.root_dir, "samsum_dataset")
        os.makedirs(self.config.root_dir, exist_ok=True)
        dataset_samsum_pt.save_to_disk(save_path)
        print(f"Tokenized dataset saved at {save_path}")


# ------------------------------
# Usage
# ------------------------------
if __name__ == "__main__":
    config = DataTransformationConfig()
    data_transformer = DataTransformation(config=config)
    data_transformer.convert()


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 12877.02 examples/s]


Tokenized dataset saved at artifacts/data_transformation\samsum_dataset
